In [1]:
# !nvidia-smi

# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
# CUDA = format_cuda_version(CUDA_version)

In [2]:
import json
import os
# import uuid  # https://docs.python.org/3/library/uuid.html
# import structlog  # for event logging
# # from dotenv import load_dotenv # enviornment vars if we want

# from pygtail import Pygtail
# import boto3
# from minio import Minio
# from dotenv import load_dotenv
from datetime import timedelta, datetime
from time import sleep
from sys import argv
# import threading
# from smart_open import smart_open
import gensim
import gensim.corpora as corpora

# setup to import the preprocessor
import sys
# from botocore.errorfactory import ClientError
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import torch_geometric
from functools import reduce
from tqdm import tqdm

from Preprocessing import Preprocessor
preprocessor = Preprocessor(0)
tqdm.pandas()


c:\Python39\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


In [3]:
joined_exists = os.path.exists('khan_joined.csv')
khan = pd.DataFrame([])
if joined_exists:
    khan = pd.read_csv('khan_joined.csv')
khan.head(5)

""


In [4]:
if not joined_exists:
    computing = pd.read_csv("Datasets\\KhanAcademy\\Computing.csv")
    computing = computing.dropna()
    computing['topic'] = 'Computing'
    computing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 263 entries, 0 to 277
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       263 non-null    object
 1   unit         263 non-null    object
 2   lesson       263 non-null    object
 3   video_title  263 non-null    object
 4   about        263 non-null    object
 5   transcript   263 non-null    object
 6   topic        263 non-null    object
dtypes: object(7)
memory usage: 16.4+ KB


In [5]:
if not joined_exists:
    economics = pd.read_csv("Datasets\\KhanAcademy\\Economics.csv")
    economics = economics.dropna()
    economics['topic'] = 'Economics'
    economics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 819 entries, 0 to 819
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       819 non-null    object
 1   unit         819 non-null    object
 2   lesson       819 non-null    object
 3   video_title  819 non-null    object
 4   about        819 non-null    object
 5   transcript   819 non-null    object
 6   topic        819 non-null    object
dtypes: object(7)
memory usage: 51.2+ KB


In [6]:
if not joined_exists:
    humanities = pd.read_csv("Datasets\\KhanAcademy\\Humanities.csv")
    humanities = humanities.dropna()
    humanities['topic'] = 'Humanities'
    humanities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1064 entries, 0 to 1152
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       1064 non-null   object
 1   unit         1064 non-null   object
 2   lesson       1064 non-null   object
 3   video_title  1064 non-null   object
 4   about        1064 non-null   object
 5   transcript   1064 non-null   object
 6   topic        1064 non-null   object
dtypes: object(7)
memory usage: 66.5+ KB


In [7]:
if not joined_exists:
    math = pd.read_csv("Datasets\\KhanAcademy\\Math.csv")
    math = math.dropna()
    math['topic'] = 'Math'
    math.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3351 entries, 0 to 3366
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       3351 non-null   object
 1   unit         3351 non-null   object
 2   lesson       3351 non-null   object
 3   video_title  3351 non-null   object
 4   about        3351 non-null   object
 5   transcript   3351 non-null   object
 6   topic        3351 non-null   object
dtypes: object(7)
memory usage: 209.4+ KB


In [8]:
if not joined_exists:
    science = pd.read_csv("Datasets\\KhanAcademy\\Science.csv")
    science = science.dropna()
    science['topic'] = 'Science'
    science.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2764 entries, 0 to 2789
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       2764 non-null   object
 1   unit         2764 non-null   object
 2   lesson       2764 non-null   object
 3   video_title  2764 non-null   object
 4   about        2764 non-null   object
 5   transcript   2764 non-null   object
 6   topic        2764 non-null   object
dtypes: object(7)
memory usage: 172.8+ KB


In [9]:
ted_main = pd.read_csv("Datasets\\TEDTalksDataset\\ted_main.csv")
transcripts = pd.read_csv("Datasets\\TEDTalksDataset\\transcripts.csv")
validation = ted_main.join(transcripts, lsuffix='url', rsuffix='url', sort=True)
validation = validation.dropna()
validation.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2466
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2461 non-null   int64 
 1   description         2461 non-null   object
 2   duration            2461 non-null   int64 
 3   event               2461 non-null   object
 4   film_date           2461 non-null   int64 
 5   languages           2461 non-null   int64 
 6   main_speaker        2461 non-null   object
 7   name                2461 non-null   object
 8   num_speaker         2461 non-null   int64 
 9   published_date      2461 non-null   int64 
 10  ratings             2461 non-null   object
 11  related_talks       2461 non-null   object
 12  speaker_occupation  2461 non-null   object
 13  tags                2461 non-null   object
 14  title               2461 non-null   object
 15  urlurl              2461 non-null   object
 16  views               2461

In [10]:
if not joined_exists:
    computing['course'].value_counts()

In [11]:
if not joined_exists:
    economics['course'].value_counts()

In [12]:
if not joined_exists:
    humanities['course'].value_counts()

In [13]:
if not joined_exists:
    math['course'].value_counts()

In [14]:
if not joined_exists:
    science['course'].value_counts()

In [15]:
if not joined_exists:
    khan_dfs = [computing, economics, humanities, math, science]
    khan = pd.concat(khan_dfs, axis=0)
    khan.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8261 entries, 0 to 2789
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   course       8261 non-null   object
 1   unit         8261 non-null   object
 2   lesson       8261 non-null   object
 3   video_title  8261 non-null   object
 4   about        8261 non-null   object
 5   transcript   8261 non-null   object
 6   topic        8261 non-null   object
dtypes: object(7)
memory usage: 516.3+ KB


In [16]:
# if not joined_exists:
# remap the courses to more broad categories: https://stackoverflow.com/a/16476974
labels = ['physics', 'chemistry', 'biology', 'algebra', 'geometry', 'statistics', 'calculus', 'history', 'economics', 'computer']
for lbl in labels:
    # print(lbl)
    for index, row in khan.iterrows():
        if lbl in row['course'].lower():
            row['course'] = lbl
khan['course'].value_counts()

algebra                                         1287
calculus                                         947
chemistry                                        909
history                                          769
biology                                          757
physics                                          743
statistics                                       626
economics                                        502
geometry                                         345
Finance and capital markets                      317
Electrical engineering                           198
Pixar in a Box                                   175
US government and civics                         139
Cosmology and astronomy                           89
AP®︎/College US Government and Politics           87
Trigonometry                                      76
computer                                          75
Differential equations                            70
Storytelling                                  

In [17]:
khan.tail(2)

,course,unit,lesson,video_title,about,transcript,topic
2788,physics,Review for AP Physics 1 exam,AP Physics 1 free response questions 2015,Question 4: 2015 AP Physics 1 free response,Identical spheres falling from the same height...,- [Voiceover] Two identical\r\nspheres are rel...,Science
2789,physics,Review for AP Physics 1 exam,AP Physics 1 free response questions 2015,Question 5: 2015 AP Physics 1 free response,Fundamental frequencies (first harmonics) of s...,- [Voiceover] The figure\r\nabove shows a stri...,Science


# Dataset Identification
It is necessary to now identify components of the dataset that can be used for the graphical machine learning. This means identifying Nodes, Edges, Node Features, and Labels. It also includes optionally including edge weights and edge features. For the sake of simplicity I think that I will be forgoing the edge weights and edge features.

I am attempting to basically do topic modeling, but without the keywords and topics that would be customary of Latent Dirchlet Allocation. Instead the goal is to train a Graph ML model using the Khan academy data.

There are two basic routes I could take. I could perform node level prediction by treating each transcript individually and seeing which is the closest match during prediction. The other option would be to perform graph level prediction by storing all of the similarly labeled transcripts together and then using the shape of the graph for comparison. 

- Nodes (Items, People, Locations, Cars, ETC)
- Edges (Connections, Interactions, Similarity, ETC)
    - Levenshtein distance over titles?
    - Number of similar named entities from SpaCy NER?
- Node Features (Attributes)
- Labels (Node-Level, Edge-Level, Graph-Level, etc)
    - I am going to first try node-level prediction as it makes more sense to me. For this I am going to use the 'course' feature in the above pandas dataframe. This will be the target that I try to predict.

In [18]:
if not joined_exists:
    # It takes 21 min to run SpaCy preprocessing over each record in the data
    khan['transcript_cleaned'] = khan['transcript'].progress_apply(lambda x: preprocessor.clean(x, fast=True))
    print(khan['transcript_cleaned'][0])

100%|██████████| 8261/8261 [00:08<00:00, 1010.66it/s]

0    hi, welcome programming! never learned program...
0    begin journey world economics, thought would b...
0    david christian: earth place humans call home....
0    - given information three triangles here. say,...
0    - hi everyone, sal khan welcome middle school ...
Name: transcript_cleaned, dtype: object


In [19]:
if not joined_exists:
    # clear up the lists to be unique only.
    khan['tokens'] = khan['transcript'].progress_apply(lambda x: preprocessor.clean(x, tokenize=True, fast=True))
    print(len(khan['tokens'][0]))
    print(khan['tokens'][0])

100%|██████████| 8261/8261 [00:08<00:00, 979.96it/s] 

5
0    [hi,, welcome, programming!, never, learned, p...
0    [begin, journey, world, economics,, thought, w...
0    [david, christian:, earth, place, humans, call...
0    [-, given, information, three, triangles, here...
0    [-, hi, everyone,, sal, khan, welcome, middle,...
Name: tokens, dtype: object


In [20]:
if not joined_exists:
# if not False:
    khan['transcript_n_entries'] = khan['tokens'].progress_apply(lambda x: len(x))
    khan.head(5)

100%|██████████| 8261/8261 [00:00<00:00, 917687.99it/s]


In [21]:
# khan = khan.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])

In [22]:
khan.head(1)

,course,unit,lesson,video_title,about,transcript,topic,transcript_cleaned,tokens,transcript_n_entries
0,computer,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"Hi, welcome to programming! If you've never le...",Computing,"hi, welcome programming! never learned program...","[hi,, welcome, programming!, never, learned, p...",183


In [23]:
if not joined_exists:
# if not False:
    khan.to_csv("khan_joined.csv", index=False)

In [24]:
# https://www.kaggle.com/code/pavansanagapati/knowledge-graph-nlp-tutorial-bert-spacy-nltk/notebook

"""
    nodes closer to yourself are more important. Take the incoming embeddings and dotproduct them with your own embedding.
    Softmax across all the dot products of the nodes coming in to have it be a probability.
    
    Need to build the corpus of all input words.
    
    Use spacy to get corpus for each doc with stopwords stripped and bigrams / trigrams joined. 
    
    Could use skipgram or bag of words with word2vec to embed words.
"""

'\n    nodes closer to yourself are more important. Take the incoming embeddings and dotproduct them with your own embedding.\n    Softmax across all the dot products of the nodes coming in to have it be a probability.\n    \n    Need to build the corpus of all input words.\n    \n    Use spacy to get corpus for each doc with stopwords stripped and bigrams / trigrams joined. \n    \n    Could use skipgram or bag of words with word2vec to embed words.\n'

In [25]:
khan.head(5)

,course,unit,lesson,video_title,about,transcript,topic,transcript_cleaned,tokens,transcript_n_entries
0,computer,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"Hi, welcome to programming! If you've never le...",Computing,"hi, welcome programming! never learned program...","[hi,, welcome, programming!, never, learned, p...",183
1,computer,Intro to JS: Drawing & Animation,Coloring,The Power of the Docs,Created by Pamela Fox.,Voiceover: Ok so you've\r\nmade a few programs...,Computing,"voiceover: ok made programs, might wondering e...","[voiceover:, ok, made, programs,, might, wonde...",542
5,computer,Intro to HTML/CSS: Making webpages,Further learning,HTML validation,Learn how to validate your webpages with the W...,"- [Voiceover] On Khan Academy, we pop up the o...",Computing,"- khan academy, pop oh noes guide tell somethi...","[-, khan, academy,, pop, oh, noes, guide, tell...",172
6,computer,Intro to SQL: Querying and managing data,SQL basics,Welcome to SQL,SQL is useful for creating and querying relati...,- [Instructor] The world is full of data. Ever...,Computing,- world full data. every app use full data. kh...,"[-, world, full, data., every, app, use, full,...",203
7,computer,Intro to SQL: Querying and managing data,SQL basics,S-Q-L or SEQUEL?,How is it pronounced? Why? Let's discuss...,"At this point, you've probably heard me\r\npro...",Computing,"point, probably heard pronounce sql two ways--...","[point,, probably, heard, pronounce, sql, two,...",129


In [26]:
khan['course'].value_counts()

algebra                                         1287
calculus                                         947
chemistry                                        909
history                                          769
biology                                          757
physics                                          743
statistics                                       626
economics                                        502
geometry                                         345
Finance and capital markets                      317
Electrical engineering                           198
Pixar in a Box                                   175
US government and civics                         139
Cosmology and astronomy                           89
AP®︎/College US Government and Politics           87
Trigonometry                                      76
computer                                          75
Differential equations                            70
Storytelling                                  

In [27]:
# khan['transcript_cleaned'] = khan['transcript']

In [28]:
# corpus = khan['tokens'].sum(axis=0)
# if isinstance(corpus, str):
#     # means it performed string concatenation so it needs to be cleaned up
#     print(corpus[0:100])
#     # corpus = str_to_list(corpus)
#     print(corpus[0: 25])
#     print(len(corpus))
#     print(type(corpus))

In [29]:
# # convert the corpus to a set since there should be no unique values
# corpus = set(corpus)
# print(len(corpus))
# corpus = list(corpus)
# print(corpus[0:25])

101429
['oscillation.', 'brawl,', 'winter.', 'snacks.', '5.28.', 'adiabatically,', 'super-intuitive', 'exponentially', 'penalized', 'validation', 'guide.', 'tackles', 'breach', 'trialscase', 'bit-zees', 'mug', 'tian,', 'patience', 'presumed', 'mid-range.', 'ese', 'kalinga.', 'spends,', 'ministers.', 'game']


In [30]:
num_topics = len(khan['course'].value_counts())
print(num_topics)

22


In [41]:
%%time
# LDA Normal
# temp = [d.split() for d in corpus]
temp = khan['tokens']
print(type(temp))
words = corpora.Dictionary(temp)
words.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
corpus = [words.doc2bow(doc) for doc in temp]

<class 'pandas.core.series.Series'>
CPU times: total: 3.92 s
Wall time: 3.93 s


In [36]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=num_topics,
                                           random_state=2,
                                           update_every=1,
                                           passes=15,
                                           alpha='auto')

CPU times: total: 1min 51s
Wall time: 1min 43s


In [37]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, words, mds='mmds', R=10)

c:\Python39\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [38]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
20    -0.238426 -0.122091       1        1  9.372458
13    -0.215129 -0.224556       2        1  7.911314
1      0.001954  0.321824       3        1  7.735497
6      0.137745  0.319859       4        1  7.686944
2      0.149548  0.146706       5        1  6.821398
8      0.373896 -0.127382       6        1  6.727304
5      0.228291  0.295431       7        1  6.569948
11    -0.318299 -0.134727       8        1  5.870019
17    -0.110389  0.152220       9        1  4.887924
10     0.106878 -0.034305      10        1  4.482201
3      0.272844  0.139349      11        1  4.480524
14    -0.279931  0.227438      12        1  4.241753
16     0.007416  0.026179      13        1  3.475332
15    -0.306025  0.093177      14        1  3.064087
21     0.025872 -0.218351      15        1  3.030401
18    -0.144992  0.304826      16        1  2.832140
0     -0.140626 -0.333793      17        1  2.812706
4     -0.371315  0.027712      18        1  2.496729
19     0.143807 -0.237659      19        1  1.638093
7      0.370263  0.032002      20        1  1.513222
9      0.027459 -0.396693      21        1  1.300752
12     0.279157 -0.257168      22        1  1.049255, topic_info=          Term          Freq         Total Category  logprob  loglift
314          x  32254.000000  32254.000000  Default  10.0000  10.0000
596      times  37611.000000  37611.000000  Default   9.0000   9.0000
2859  negative  28542.000000  28542.000000  Default   8.0000   8.0000
1066     minus  23899.000000  23899.000000  Default   7.0000   7.0000
668       plus  25072.000000  25072.000000  Default   6.0000   6.0000
...        ...           ...           ...      ...      ...      ...
2331      hand    773.763710   2022.695017  Topic22  -3.9168   3.5962
957       wire    475.669873   1125.682930  Topic22  -4.4033   3.6957
1932   current    632.826972   2063.248444  Topic22  -4.1178   3.3753
166       draw    627.501567   9812.731050  Topic22  -4.1263   1.8074
231      maybe    452.689418  10482.739130  Topic22  -4.4528   1.4148

[528 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
7523       3  0.999138       $1
7258       1  0.056950        .
7258       2  0.007119        .
7258       7  0.010678        .
7258       8  0.088984        .
...      ...       ...      ...
984       15  0.002910    zero.
984       17  0.045111    zero.
984       19  0.004948    zero.
984       20  0.029977    zero.
12743      4  0.997597  zucker:

[2110 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[21, 14, 2, 7, 3, 9, 6, 12, 18, 11, 4, 15, 17, 16, 22, 19, 1, 5, 20, 8, 10, 13])

In [39]:
validation.head(5)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,urlurl,views,transcript,urlurl
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...","[{'id': 865, 'hero': 'https://pe.tedcdn.com/im...",Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...","[{'id': 243, 'hero': 'https://pe.tedcdn.com/im...",Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...","[{'id': 1725, 'hero': 'https://pe.tedcdn.com/i...",Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","[{'id': 1041, 'hero': 'https://pe.tedcdn.com/i...",Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...","[{'id': 2056, 'hero': 'https://pe.tedcdn.com/i...",Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [40]:
new_text_corpus = words.doc2bow(validation['transcript'][0].split())
print(len(new_text_corpus))
prediction = lda_model.get_document_topics(new_text_corpus)
prediction.sort(key = lambda x: x[1], reverse = True)
print(prediction)
prediction = prediction[0][0]
print("Predicted Topic: %d" % prediction)
lda_model.show_topic(3)

621
[(5, 0.6481136), (6, 0.22089048), (17, 0.059130847), (3, 0.030549446), (10, 0.0153239565)]
Predicted Topic: 5


[('use', 0.007523589),
 ('kind', 0.0064453157),
 ('around', 0.006415009),
 ('need', 0.0059491163),
 ('things', 0.0058866753),
 ('got', 0.005176398),
 ('put', 0.0048908405),
 ('back', 0.0048693996),
 ('called', 0.004515075),
 ('earth', 0.0044959947)]